In [ ]:
from pythonnet import load
load("coreclr")

import os
import clr
clr.AddReference(os.path.abspath("../src/DerivaSharp/bin/Release/net10.0/win-x64/publish/DerivaSharp.dll"))

import matplotlib.pyplot as plt
import numpy as np

from DerivaSharp.Instruments import BarrierTouchStatus, SnowballOption
from DerivaSharp.PricingEngines import FdSnowballEngine, FiniteDifferenceScheme, McSnowballEngine, PricingContext
from DerivaSharp.Time import DateUtils
from System import DateOnly

In [ ]:
effectiveDate = DateOnly(2022, 1, 5)
expirationDate = DateOnly(2023, 1, 5)
lockUpMonths = 3
koObsDates = list(DateUtils.GetObservationDates(effectiveDate, expirationDate, lockUpMonths))

for date in koObsDates:
    print(date.ToString())

In [ ]:
coupon = 0.085
ki = 0.8
ko = 1.03
strike = 1.0
vol = 0.16
r = 0.02
q = 0.04

option = SnowballOption.CreateStandardSnowball(
    coupon,
    strike,
    ki,
    ko,
    koObsDates,
    BarrierTouchStatus.NoTouch,
    effectiveDate,
    expirationDate,
)

fd_engine = FdSnowballEngine(FiniteDifferenceScheme.CrankNicolson, 1000, 500)
mc_engine = McSnowballEngine(100000, True)

In [ ]:
valuationDate = DateOnly(2022, 1, 5)
spots = np.arange(0.5, 1.51, 0.01, dtype=np.float64)
ctx = PricingContext(1, valuationDate, vol, r, q)

fd_values = -np.array(fd_engine.Values(option, ctx, spots))
fd_deltas = -np.array(fd_engine.Deltas(option, ctx, spots))
fd_gammas = -np.array(fd_engine.Gammas(option, ctx, spots))

mc_values = -np.array(mc_engine.Values(option, ctx, spots))
mc_deltas = -np.array(mc_engine.Deltas(option, ctx, spots))
mc_gammas = -np.array(mc_engine.Gammas(option, ctx, spots))

In [ ]:
plt.rcParams["figure.figsize"] = [10, 12]
plt.rcParams["axes.grid"] = True
plt.rcParams["grid.alpha"] = 0.3

fig, axs = plt.subplots(3, 1, sharex=True)

metrics = [("Value", fd_values, mc_values), ("Delta", fd_deltas, mc_deltas), ("Gamma", fd_gammas, mc_gammas)]

for i, (name, fd_data, mc_data) in enumerate(metrics):
    ax = axs[i]

    (l1,) = ax.plot(spots, fd_data, label="Finite Difference", linewidth=2)
    (l2,) = ax.plot(spots, mc_data, label="Monte Carlo", linestyle="--", linewidth=2)

    ax.set_ylabel(name, fontsize=12, fontweight="bold")
    ax.set_title(f"Snowball Option {name}", fontsize=14)

    ax_diff = ax.twinx()
    diff = fd_data - mc_data
    bars = ax_diff.bar(spots, diff, width=0.01, color="green", alpha=0.2, label="Diff (FD - MC)")
    ax_diff.set_ylabel("Difference", color="green")
    ax_diff.tick_params(axis="y", labelcolor="green")

    lines = [l1, l2, bars]
    labels = [l.get_label() for l in lines]
    ax.legend(lines, labels, loc="upper left", frameon=True)

axs[-1].set_xlabel("Spot Price", fontsize=12, fontweight="bold")
plt.tight_layout()
plt.show()